In [27]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [28]:
titanic = sns.load_dataset("titanic")

In [29]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [30]:
print("Shape:", titanic.shape)
print("Columns:", titanic.columns.tolist())

Shape: (891, 15)
Columns: ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone']


In [31]:
df = titanic[['sex', 'class', 'embark_town', 'alone', 'survived']].dropna()

In [32]:
df.shape

(889, 5)

In [33]:
# Encode categorical variables

label_encoders = {}
for col in ['sex', 'class', 'embark_town', 'alone']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [34]:
label_encoders

{'sex': LabelEncoder(),
 'class': LabelEncoder(),
 'embark_town': LabelEncoder(),
 'alone': LabelEncoder()}

In [35]:
X = df.drop('survived', axis=1)
y = df['survived']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [38]:
cat_nb = CategoricalNB()

In [39]:
cat_nb.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None
,min_categories,None


In [40]:
y_train_pred = cat_nb.predict(X_train)
y_test_pred = cat_nb.predict(X_test)

In [41]:
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

Train Accuracy: 0.7876230661040787
Test Accuracy: 0.7808988764044944


In [42]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Confusion Matrix:
 [[93 17]
 [22 46]]


In [43]:
print("Classification Report:\n", classification_report(y_test, y_test_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.85      0.83       110
           1       0.73      0.68      0.70        68

    accuracy                           0.78       178
   macro avg       0.77      0.76      0.76       178
weighted avg       0.78      0.78      0.78       178



In [44]:
cv_scores = cross_val_score(cat_nb, X, y, cv=5, scoring='accuracy')

In [45]:
cv_scores

array([0.66292135, 0.78651685, 0.81460674, 0.75280899, 0.78531073])

In [46]:
print("Cross-Validation Accuracy (5-fold):", np.mean(cv_scores).round(4))

Cross-Validation Accuracy (5-fold): 0.7604


# Prediction on sample

In [47]:
sex_cnb = "female"
class_cnb = "First"
embark_town_cnb = "Southampton"
alone_cnb = False

In [48]:
sample = pd.DataFrame({
    'sex': [label_encoders['sex'].transform([sex_cnb])[0]],
    'class': [label_encoders['class'].transform([class_cnb])[0]],
    'embark_town': [label_encoders['embark_town'].transform([embark_town_cnb])[0]],
    'alone': [label_encoders['alone'].transform([alone_cnb])[0]]
})

In [49]:
prediction = cat_nb.predict(sample)
prediction

array([1])

In [50]:
prediction = cat_nb.predict(sample)[0]
prediction

np.int64(1)

In [24]:
sample

,sex,class,embark_town,alone
0,0,0,2,0


In [51]:
print("Predicted Survival:", "Survived" if prediction == 1 else "Did not survive")

Predicted Survival: Survived
